In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
source(paste0(getwd(), "/src/currents.R"))

R[write to console]: data.table 1.14.2 using 4 threads (see ?getDTthreads).  Latest news: r-datatable.com

R[write to console]: Loading required package: MASS

R[write to console]: Loading required package: strucchange

R[write to console]: Loading required package: zoo

R[write to console]: 
Attaching package: ‘zoo’


R[write to console]: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


R[write to console]: Loading required package: sandwich

R[write to console]: Loading required package: urca

R[write to console]: Loading required package: lmtest

R[write to console]: 
Attaching package: ‘dbscan’


R[write to console]: The following object is masked from ‘package:stats’:

    as.dendrogram




In [3]:
%%R
tab <- get_data_on_currents(paste0(getwd(), '/data/cm2011a1_87364.dat'))
tab_split <- split_currents_data(tab$tab_multi_years)

Оценим качество бейзлайна.

In [4]:
%%R
train_res <- baseline_model_on_train(tab_split$train)
test_res <- baseline_model_on_test(tab_split$test, tab_split$train)
print(calculate_statistics(train_res))
print(calculate_statistics(test_res))

[1] 0.555367
[1] 0.5117201


Мы видим, что судя по вариации sMAPE бейзлайн является моделью на грани плохой и нормальной.

Оцени подход с catboost на данных hdbscan и sma

In [5]:
%%R
tab_train <- prepare_train_data(tab_split$train)
tab_sma <- sma_model_for_train(tab_split$train)
tab_sma <- tab_sma[, list(ID, sma_U, sma_V)]
setkey(tab_train, "i.ID")
setkey(tab_sma, "ID")
tab_train <- tab_sma[tab_train]

In [6]:
%%R
library(data.table)
fwrite(tab_train, "train_dataset_currents.csv")

In [7]:
import pandas as pd

In [8]:
tab_train = pd.read_csv("train_dataset_currents.csv")

In [9]:
tab_train['horizont_years'] = tab_train['year_of_initial_deployment'] - tab_train['i.year_of_initial_deployment']
tab_train['horizont_months'] = tab_train['number_of_months_of_deployment'] - tab_train['i.number_of_months_of_deployment'] 

In [10]:
tab_train['horizont_years'].describe(), tab_train['horizont_months'].describe()

(count    128.000000
 mean       0.171875
 std        1.004787
 min        0.000000
 25%        0.000000
 50%        0.000000
 75%        0.000000
 max        9.000000
 Name: horizont_years, dtype: float64,
 count    128.000000
 mean       3.898438
 std        6.085791
 min      -14.000000
 25%        1.000000
 50%        2.000000
 75%        5.000000
 max       54.000000
 Name: horizont_months, dtype: float64)

Мы видим, что прогноз делается в среднем на 2 месяца.

In [11]:
(tab_train[(tab_train['horizont_months'] <= 0) & (tab_train['horizont_years'] <= 0)])

,ID,sma_U,sma_V,id,year_of_initial_deployment,number_of_months_of_deployment,instrument_depth,mean_U,mean_V,std_dev_U,...,i.latitude,i.longitude,i.number_of_years,i.max_id,i.min_id,i.cluster,i.mean_cluster_U,i.mean_cluster_V,horizont_years,horizont_months


Данные корректны, в будущее мы не заглядываем.

In [12]:
from catboost import CatBoostRegressor

In [13]:
model_1 = CatBoostRegressor(iterations=1000)
model_2 = CatBoostRegressor(iterations=1000)

In [14]:
tab_train.columns

Index(['ID', 'sma_U', 'sma_V', 'id', 'year_of_initial_deployment',
       'number_of_months_of_deployment', 'instrument_depth', 'mean_U',
       'mean_V', 'std_dev_U', 'std_dev_V', 'nominal_depth', 'latitude',
       'longitude', 'i.ID', 'number_of_years', 'max_id', 'min_id', 'cluster',
       'mean_cluster_U', 'mean_cluster_V', 'i.year_of_initial_deployment',
       'i.number_of_months_of_deployment', 'i.instrument_depth', 'i.mean_U',
       'i.mean_V', 'i.std_dev_U', 'i.std_dev_V', 'i.nominal_depth',
       'i.latitude', 'i.longitude', 'i.number_of_years', 'i.max_id',
       'i.min_id', 'i.cluster', 'i.mean_cluster_U', 'i.mean_cluster_V',
       'horizont_years', 'horizont_months'],
      dtype='object')

In [15]:
X = tab_train.copy()
X.drop(['id', 'ID', 'max_id', 'min_id', 'i.ID', 'mean_U', 'mean_V', 'std_dev_U', 'std_dev_V', 'i.number_of_years',
       'i.max_id', 'i.min_id', 'number_of_years', 'mean_cluster_U', 'mean_cluster_V'], axis = 1, inplace = True)
y1 = tab_train[['mean_U']]
y2 = tab_train[['mean_V']]

In [16]:
model_1.fit(X, y1, verbose = False)
model_2.fit(X, y2, verbose = False)

In [17]:
pred1 = model_1.predict(X)
pred2 = model_2.predict(X)
predictions = pd.DataFrame({'pred1': pred1, 'pred2': pred2, 'target1': y1['mean_U'], 'target2': y2['mean_V']})

In [18]:
(sum(abs(predictions['pred1'] - predictions['target1'])/(abs(predictions['pred1']) + abs(predictions['target1']))) +\
sum(abs(predictions['pred2'] - predictions['target2'])/(abs(predictions['pred2']) + abs(predictions['target2']))))/(2*len(predictions))

0.0785594917580592

In [19]:
%%R
tab_test <- prepare_test_data(tab_split$test, tab_split$train)
tab_sma <- sma_model_for_train(tab_split$train)
tab_sma <- tab_sma[, list(ID, sma_U, sma_V)]
setkey(tab_test, "i.ID")
setkey(tab_sma, "ID")
tab_test <- tab_sma[tab_test]

In [20]:
%%R
fwrite(tab_test, "test_dataset_currents.csv")

In [21]:
tab_test = pd.read_csv("test_dataset_currents.csv")

In [22]:
tab_test['horizont_years'] = tab_test['year_of_initial_deployment'] - tab_test['i.year_of_initial_deployment']
tab_test['horizont_months'] = tab_test['number_of_months_of_deployment'] - tab_test['i.number_of_months_of_deployment'] 

In [23]:
tab_test['horizont_years'].describe(), tab_test['horizont_months'].describe() 

(count    308.000000
 mean       0.081169
 std        0.611962
 min        0.000000
 25%        0.000000
 50%        0.000000
 75%        0.000000
 max       10.000000
 Name: horizont_years, dtype: float64,
 count    308.00000
 mean       4.38961
 std        5.01634
 min      -11.00000
 25%        1.00000
 50%        3.00000
 75%        7.00000
 max       37.00000
 Name: horizont_months, dtype: float64)

Мы видим, что прогноз делается в среднем на 3 месяца.

In [24]:
(tab_test[(tab_test['horizont_months'] <= 0) & (tab_test['horizont_years'] <= 0)])

,ID,sma_U,sma_V,id,year_of_initial_deployment,number_of_months_of_deployment,instrument_depth,mean_U,mean_V,std_dev_U,...,i.latitude,i.longitude,i.number_of_years,i.max_id,i.min_id,i.cluster,i.mean_cluster_U,i.mean_cluster_V,horizont_years,horizont_months


Мы видим, что данные корректны. В будущее мы не заглядываем.

In [25]:
X = tab_test.copy()
X.drop(['ID', 'id', 'max_id', 'min_id', 'i.ID', 'mean_U', 'mean_V', 'std_dev_U', 'std_dev_V', 'i.number_of_years',
       'i.max_id', 'i.min_id', 'number_of_years', 'mean_cluster_U', 'mean_cluster_V'], axis = 1, inplace = True)
y1 = tab_test[['mean_U']]
y2 = tab_test[['mean_V']]

In [26]:
tab_train.columns

Index(['ID', 'sma_U', 'sma_V', 'id', 'year_of_initial_deployment',
       'number_of_months_of_deployment', 'instrument_depth', 'mean_U',
       'mean_V', 'std_dev_U', 'std_dev_V', 'nominal_depth', 'latitude',
       'longitude', 'i.ID', 'number_of_years', 'max_id', 'min_id', 'cluster',
       'mean_cluster_U', 'mean_cluster_V', 'i.year_of_initial_deployment',
       'i.number_of_months_of_deployment', 'i.instrument_depth', 'i.mean_U',
       'i.mean_V', 'i.std_dev_U', 'i.std_dev_V', 'i.nominal_depth',
       'i.latitude', 'i.longitude', 'i.number_of_years', 'i.max_id',
       'i.min_id', 'i.cluster', 'i.mean_cluster_U', 'i.mean_cluster_V',
       'horizont_years', 'horizont_months'],
      dtype='object')

In [27]:
tab_test.columns

Index(['ID', 'sma_U', 'sma_V', 'id', 'year_of_initial_deployment',
       'number_of_months_of_deployment', 'instrument_depth', 'mean_U',
       'mean_V', 'std_dev_U', 'std_dev_V', 'nominal_depth', 'latitude',
       'longitude', 'i.ID', 'number_of_years', 'max_id', 'min_id', 'cluster',
       'mean_cluster_U', 'mean_cluster_V', 'i.year_of_initial_deployment',
       'i.number_of_months_of_deployment', 'i.instrument_depth', 'i.mean_U',
       'i.mean_V', 'i.std_dev_U', 'i.std_dev_V', 'i.nominal_depth',
       'i.latitude', 'i.longitude', 'i.number_of_years', 'i.max_id',
       'i.min_id', 'i.cluster', 'i.mean_cluster_U', 'i.mean_cluster_V',
       'horizont_years', 'horizont_months'],
      dtype='object')

In [28]:
pred1 = model_1.predict(X)
pred2 = model_2.predict(X)
predictions = pd.DataFrame({'pred1': pred1, 'pred2': pred2, 'target1': y1['mean_U'], 'target2': y2['mean_V']})

In [29]:
(sum(abs(predictions['pred1'] - predictions['target1'])/(abs(predictions['pred1']) + abs(predictions['target1']))) +\
sum(abs(predictions['pred2'] - predictions['target2'])/(abs(predictions['pred2']) + abs(predictions['target2']))))/(2*len(predictions))

0.574056264593503

Как видно, catboost бейзлайн на тестовом множестве не пробил.

In [30]:
X = tab_train.copy()
X.drop(['id', 'ID', 'max_id', 'min_id', 'i.ID', 'mean_U', 'mean_V', 'std_dev_U', 'std_dev_V', 'i.number_of_years',
       'i.max_id', 'i.min_id', 'number_of_years'], axis = 1, inplace = True)
y1 = tab_train[['mean_U']]
y2 = tab_train[['mean_V']]

In [31]:
_ = [print(col, y1['mean_U'].corr(X[col])) for col in X.columns]

sma_U 0.5646726770123857
sma_V 0.005329460875333909
year_of_initial_deployment 0.13607915375093124
number_of_months_of_deployment 0.13469040077895866
instrument_depth -0.014541913410564163
nominal_depth -0.17145571734873535
latitude 0.3631555190897275
longitude -0.16270064830435724
cluster -0.2714193583814469
mean_cluster_U 0.6490760707146358
mean_cluster_V 0.08035126336793638
i.year_of_initial_deployment 0.13599780426122285
i.number_of_months_of_deployment 0.1372815625605336
i.instrument_depth -0.17929153535792244
i.mean_U 0.5649452351647541
i.mean_V 0.00023716917635121222
i.std_dev_U 0.022109164544043023
i.std_dev_V -0.0026858446499959305
i.nominal_depth -0.17145571734873535
i.latitude 0.3631555190897275
i.longitude -0.16270064830435724
i.cluster -0.2714193583814469
i.mean_cluster_U 0.6085711020362163
i.mean_cluster_V 0.053228845600675576
horizont_years 0.14636477549423874
horizont_months 0.04764706244025385


In [32]:
_ = [print(col, y2['mean_V'].corr(X[col])) for col in X.columns]

sma_U -0.12178325273986829
sma_V 0.652429300623904
year_of_initial_deployment 0.19257110742403274
number_of_months_of_deployment -0.10543158972780058
instrument_depth -0.05919397457634953
nominal_depth -0.1833148444095521
latitude -0.14891960477161603
longitude -0.09440263215387729
cluster 0.14092169284053033
mean_cluster_U 0.16969457344469652
mean_cluster_V 0.6928400320918733
i.year_of_initial_deployment 0.19259416523622136
i.number_of_months_of_deployment -0.09306472084010423
i.instrument_depth -0.15662028243920492
i.mean_U -0.12109739181942297
i.mean_V 0.6551253369567122
i.std_dev_U 0.1978061942526405
i.std_dev_V 0.3221593077545234
i.nominal_depth -0.1833148444095521
i.latitude -0.14891960477161603
i.longitude -0.09440263215387729
i.cluster 0.14092169284053033
i.mean_cluster_U 0.12539090846028123
i.mean_cluster_V 0.6753220260447823
horizont_years 0.022890874490810138
horizont_months -0.057443235937824036


Анализ корреляций показывают, что тут подойдут стандартные модели из временных рядов.

Попробуем ещё раз бустинг, но для урезанного количества переменных.

In [33]:
model_1 = CatBoostRegressor(iterations=1000)
model_2 = CatBoostRegressor(iterations=1000)

In [34]:
X = tab_train.copy()
X = X[['nominal_depth', 'horizont_years', 'horizont_months', 'i.mean_U', 'i.mean_V', 'i.std_dev_U', 'i.std_dev_V', 'i.nominal_depth', 'i.mean_cluster_U', 'i.mean_cluster_V', 'sma_U', 'sma_V']]
y1 = tab_train[['mean_U']]
y2 = tab_train[['mean_V']]

In [35]:
model_1.fit(X, y1, verbose = False)
model_2.fit(X, y2, verbose = False)

In [36]:
pred1 = model_1.predict(X)
pred2 = model_2.predict(X)
predictions = pd.DataFrame({'pred1': pred1, 'pred2': pred2, 'target1': y1['mean_U'], 'target2': y2['mean_V']})

In [37]:
(sum(abs(predictions['pred1'] - predictions['target1'])/(abs(predictions['pred1']) + abs(predictions['target1']))) +\
sum(abs(predictions['pred2'] - predictions['target2'])/(abs(predictions['pred2']) + abs(predictions['target2']))))/(2*len(predictions))

0.12810462929737068

In [38]:
X = tab_test.copy()
X = X[['nominal_depth', 'horizont_years', 'horizont_months', 'i.mean_U', 'i.mean_V', 'i.std_dev_U', 'i.std_dev_V', 'i.nominal_depth', 'i.mean_cluster_U', 'i.mean_cluster_V', 'sma_U', 'sma_V']]
y1 = tab_test[['mean_U']]
y2 = tab_test[['mean_V']]

In [39]:
pred1 = model_1.predict(X)
pred2 = model_2.predict(X)
predictions = pd.DataFrame({'pred1': pred1, 'pred2': pred2, 'target1': y1['mean_U'], 'target2': y2['mean_V']})

In [40]:
(sum(abs(predictions['pred1'] - predictions['target1'])/(abs(predictions['pred1']) + abs(predictions['target1']))) +\
sum(abs(predictions['pred2'] - predictions['target2'])/(abs(predictions['pred2']) + abs(predictions['target2']))))/(2*len(predictions))

0.5628490466817321

Как видно, урезание переменных помогло улучшить качество бустинга, но бейзлайн всё ещё не пробит.

Попробуем модель, являющуюся взвешенной комбинацией бейзлайна и HDBSCAN.

In [41]:
%%R
train_res_weighted <- weighted_model_on_train(tab_split$train, 0.8)
calculate_statistics(train_res_weighted)

[1] 0.5439477


In [42]:
%%R
test_res_weighted <- weighted_model_on_test(tab_split$test, tab_split$train, 0.8)
calculate_statistics(test_res_weighted)

[1] 0.5017438


Как видно, бейзлайн пробит.

Попробуем EMA модель.

In [43]:
%%R
train_res_ema <- ema_model_for_train(tab_split$train, 2)
calculate_statistics(train_res_ema)

[1] 0.5580376


In [44]:
%%R
test_res_ema <- ema_model_for_test(tab_split$test, tab_split$train)
calculate_statistics(test_res_ema)

[1] 0.503738


Как видно, бейзлайн пробит.

Попробуем интерпретируемую модель, вводящую в бейзлайн ещё один фактор с весом.

In [45]:
%%R
train_res <- interpretable_model_on_train(tab_split$train, 0.025)
calculate_statistics(na.omit(train_res))

[1] 0.5339282


In [46]:
%%R
test_res <- interpretable_model_on_test(tab_split$test, tab_split$train)
calculate_statistics(test_res)

[1] 0.5145549


Как видно, бейзлайн не пробит.

Попробуем VAR модель.

In [47]:
%%R
train_res <- var_model_for_train(tab_split$train)
calculate_statistics(na.omit(train_res))

[1] 0.2851082


In [48]:
%%R
test_res <- var_model_for_test(tab_split$test, tab_split$train)
calculate_statistics(test_res)

[1] 0.5229228


Как видно, бейзлайн не пробит.

Попробуем SMA модель.

In [49]:
predictions = pd.DataFrame({'ID': tab_train['ID'],
                            'pred1': tab_train['sma_U'], 'pred2': tab_train['sma_V'],
                            'target1': tab_train['mean_U'], 'target2': tab_train['mean_V']})

In [50]:
(sum(abs(predictions['pred1'] - predictions['target1'])/(abs(predictions['pred1']) + abs(predictions['target1']))) +\
sum(abs(predictions['pred2'] - predictions['target2'])/(abs(predictions['pred2']) + abs(predictions['target2']))))/(2*len(predictions))

0.5642800692257919

In [51]:
predictions = pd.DataFrame({'ID': tab_test['ID'],
                            'pred1': tab_test['sma_U'], 'pred2': tab_test['sma_V'],
                            'target1': tab_test['mean_U'], 'target2': tab_test['mean_V']})

In [52]:
(sum(abs(predictions['pred1'] - predictions['target1'])/(abs(predictions['pred1']) + abs(predictions['target1']))) +\
sum(abs(predictions['pred2'] - predictions['target2'])/(abs(predictions['pred2']) + abs(predictions['target2']))))/(2*len(predictions))

0.5136999095421154

Как видно бейзлайн не пробит.

Мы рассмотрим два варианта блендинга. 
Первый вариант: блендинг взвешенной модели м EMA-модели, а потом блендинг результата с лучшим найденным catboost.

In [53]:
%%R
train_blended <- na.omit(blend_two_models(train_res_weighted, train_res_ema))
print(calculate_statistics(train_blended))
fwrite(train_blended, "blending_two_models_train.csv")

[1] 0.5509774


In [54]:
%%R
test_blended <- na.omit(blend_two_models(test_res_weighted, test_res_ema))
print(calculate_statistics(test_blended))
fwrite(test_blended, "blending_two_models_test.csv")

[1] 0.4998604


In [55]:
train_blended = pd.read_csv('blending_two_models_train.csv')
test_blended = pd.read_csv('blending_two_models_test.csv')

In [56]:
model_1 = CatBoostRegressor(iterations=1000)
model_2 = CatBoostRegressor(iterations=1000)

In [57]:
X = tab_train.copy()
X = X[['nominal_depth', 'horizont_years', 'horizont_months', 'i.mean_U', 'i.mean_V', 'i.std_dev_U', 'i.std_dev_V', 'i.nominal_depth', 'i.mean_cluster_U', 'i.mean_cluster_V', 'sma_U', 'sma_V']]
y1 = tab_train[['mean_U']]
y2 = tab_train[['mean_V']]

In [58]:
model_1.fit(X, y1)
model_2.fit(X, y2)

Learning rate set to 0.029588
0:	learn: 7.1765001	total: 492us	remaining: 492ms
1:	learn: 7.1082730	total: 871us	remaining: 435ms
2:	learn: 7.0395415	total: 1.24ms	remaining: 412ms
3:	learn: 6.9811875	total: 1.64ms	remaining: 408ms
4:	learn: 6.9200560	total: 2.01ms	remaining: 401ms
5:	learn: 6.8582311	total: 2.37ms	remaining: 393ms
6:	learn: 6.8076797	total: 2.74ms	remaining: 388ms
7:	learn: 6.7455404	total: 3.08ms	remaining: 382ms
8:	learn: 6.6977650	total: 3.43ms	remaining: 377ms
9:	learn: 6.6477293	total: 3.76ms	remaining: 372ms
10:	learn: 6.5860134	total: 4.11ms	remaining: 369ms
11:	learn: 6.5179604	total: 4.43ms	remaining: 365ms
12:	learn: 6.4671458	total: 4.76ms	remaining: 361ms
13:	learn: 6.4158410	total: 5.09ms	remaining: 359ms
14:	learn: 6.3675456	total: 5.44ms	remaining: 357ms
15:	learn: 6.3246135	total: 5.76ms	remaining: 354ms
16:	learn: 6.2701004	total: 6.07ms	remaining: 351ms
17:	learn: 6.2278063	total: 6.42ms	remaining: 350ms
18:	learn: 6.1716242	total: 6.74ms	remaining: 

357:	learn: 1.4047860	total: 110ms	remaining: 197ms
358:	learn: 1.4024730	total: 110ms	remaining: 197ms
359:	learn: 1.3967786	total: 111ms	remaining: 197ms
360:	learn: 1.3935768	total: 111ms	remaining: 196ms
361:	learn: 1.3918831	total: 111ms	remaining: 196ms
362:	learn: 1.3819994	total: 112ms	remaining: 196ms
363:	learn: 1.3803348	total: 112ms	remaining: 196ms
364:	learn: 1.3740249	total: 112ms	remaining: 196ms
365:	learn: 1.3726383	total: 113ms	remaining: 195ms
366:	learn: 1.3696386	total: 113ms	remaining: 195ms
367:	learn: 1.3613090	total: 113ms	remaining: 195ms
368:	learn: 1.3546613	total: 114ms	remaining: 194ms
369:	learn: 1.3517098	total: 114ms	remaining: 194ms
370:	learn: 1.3503560	total: 114ms	remaining: 194ms
371:	learn: 1.3489446	total: 115ms	remaining: 194ms
372:	learn: 1.3475328	total: 115ms	remaining: 193ms
373:	learn: 1.3412187	total: 115ms	remaining: 193ms
374:	learn: 1.3361913	total: 116ms	remaining: 193ms
375:	learn: 1.3349052	total: 116ms	remaining: 192ms
376:	learn: 

528:	learn: 0.8895535	total: 162ms	remaining: 144ms
529:	learn: 0.8890429	total: 162ms	remaining: 144ms
530:	learn: 0.8877226	total: 163ms	remaining: 144ms
531:	learn: 0.8838563	total: 163ms	remaining: 144ms
532:	learn: 0.8833484	total: 164ms	remaining: 143ms
533:	learn: 0.8804468	total: 164ms	remaining: 143ms
534:	learn: 0.8792507	total: 164ms	remaining: 143ms
535:	learn: 0.8787742	total: 165ms	remaining: 143ms
536:	learn: 0.8783542	total: 165ms	remaining: 142ms
537:	learn: 0.8753712	total: 165ms	remaining: 142ms
538:	learn: 0.8730290	total: 166ms	remaining: 142ms
539:	learn: 0.8723822	total: 166ms	remaining: 141ms
540:	learn: 0.8686123	total: 166ms	remaining: 141ms
541:	learn: 0.8670853	total: 167ms	remaining: 141ms
542:	learn: 0.8666321	total: 167ms	remaining: 140ms
543:	learn: 0.8643297	total: 167ms	remaining: 140ms
544:	learn: 0.8618530	total: 167ms	remaining: 140ms
545:	learn: 0.8578609	total: 168ms	remaining: 140ms
546:	learn: 0.8550429	total: 168ms	remaining: 139ms
547:	learn: 

984:	learn: 0.2875985	total: 296ms	remaining: 4.51ms
985:	learn: 0.2865907	total: 297ms	remaining: 4.21ms
986:	learn: 0.2856178	total: 297ms	remaining: 3.91ms
987:	learn: 0.2850651	total: 297ms	remaining: 3.61ms
988:	learn: 0.2846434	total: 298ms	remaining: 3.31ms
989:	learn: 0.2844894	total: 298ms	remaining: 3.01ms
990:	learn: 0.2836568	total: 298ms	remaining: 2.71ms
991:	learn: 0.2828691	total: 299ms	remaining: 2.41ms
992:	learn: 0.2818517	total: 299ms	remaining: 2.11ms
993:	learn: 0.2817475	total: 299ms	remaining: 1.8ms
994:	learn: 0.2811715	total: 299ms	remaining: 1.5ms
995:	learn: 0.2806253	total: 300ms	remaining: 1.2ms
996:	learn: 0.2798523	total: 300ms	remaining: 902us
997:	learn: 0.2790720	total: 300ms	remaining: 601us
998:	learn: 0.2783752	total: 301ms	remaining: 300us
999:	learn: 0.2777012	total: 301ms	remaining: 0us
Learning rate set to 0.029588
0:	learn: 6.1173582	total: 404us	remaining: 404ms
1:	learn: 6.0479891	total: 616us	remaining: 308ms
2:	learn: 5.9756518	total: 933u

193:	learn: 1.7074438	total: 59.7ms	remaining: 248ms
194:	learn: 1.7010341	total: 60.1ms	remaining: 248ms
195:	learn: 1.6958058	total: 60.4ms	remaining: 248ms
196:	learn: 1.6900890	total: 60.7ms	remaining: 247ms
197:	learn: 1.6842910	total: 61ms	remaining: 247ms
198:	learn: 1.6768572	total: 61.3ms	remaining: 247ms
199:	learn: 1.6696176	total: 61.6ms	remaining: 246ms
200:	learn: 1.6617040	total: 61.9ms	remaining: 246ms
201:	learn: 1.6565385	total: 62.2ms	remaining: 246ms
202:	learn: 1.6506120	total: 62.5ms	remaining: 245ms
203:	learn: 1.6469384	total: 62.8ms	remaining: 245ms
204:	learn: 1.6412503	total: 63.1ms	remaining: 245ms
205:	learn: 1.6337899	total: 63.3ms	remaining: 244ms
206:	learn: 1.6266433	total: 63.6ms	remaining: 244ms
207:	learn: 1.6200272	total: 63.9ms	remaining: 243ms
208:	learn: 1.6135203	total: 64.2ms	remaining: 243ms
209:	learn: 1.6081592	total: 64.5ms	remaining: 243ms
210:	learn: 1.6029385	total: 64.8ms	remaining: 242ms
211:	learn: 1.5965658	total: 65.1ms	remaining: 2

569:	learn: 0.5810402	total: 170ms	remaining: 128ms
570:	learn: 0.5787034	total: 171ms	remaining: 128ms
571:	learn: 0.5761979	total: 171ms	remaining: 128ms
572:	learn: 0.5751163	total: 171ms	remaining: 128ms
573:	learn: 0.5748890	total: 171ms	remaining: 127ms
574:	learn: 0.5746118	total: 172ms	remaining: 127ms
575:	learn: 0.5744088	total: 172ms	remaining: 127ms
576:	learn: 0.5742028	total: 172ms	remaining: 126ms
577:	learn: 0.5722064	total: 173ms	remaining: 126ms
578:	learn: 0.5704546	total: 173ms	remaining: 126ms
579:	learn: 0.5684011	total: 173ms	remaining: 125ms
580:	learn: 0.5664179	total: 174ms	remaining: 125ms
581:	learn: 0.5661990	total: 174ms	remaining: 125ms
582:	learn: 0.5641748	total: 174ms	remaining: 125ms
583:	learn: 0.5626993	total: 174ms	remaining: 124ms
584:	learn: 0.5609244	total: 175ms	remaining: 124ms
585:	learn: 0.5579363	total: 175ms	remaining: 124ms
586:	learn: 0.5571071	total: 175ms	remaining: 123ms
587:	learn: 0.5568659	total: 176ms	remaining: 123ms
588:	learn: 

745:	learn: 0.3716487	total: 222ms	remaining: 75.6ms
746:	learn: 0.3708352	total: 222ms	remaining: 75.4ms
747:	learn: 0.3692003	total: 223ms	remaining: 75.1ms
748:	learn: 0.3690260	total: 223ms	remaining: 74.8ms
749:	learn: 0.3685613	total: 223ms	remaining: 74.5ms
750:	learn: 0.3669717	total: 224ms	remaining: 74.2ms
751:	learn: 0.3656937	total: 224ms	remaining: 73.9ms
752:	learn: 0.3645866	total: 224ms	remaining: 73.6ms
753:	learn: 0.3631000	total: 225ms	remaining: 73.4ms
754:	learn: 0.3622879	total: 225ms	remaining: 73.1ms
755:	learn: 0.3616404	total: 225ms	remaining: 72.8ms
756:	learn: 0.3600490	total: 226ms	remaining: 72.5ms
757:	learn: 0.3592985	total: 226ms	remaining: 72.2ms
758:	learn: 0.3576684	total: 226ms	remaining: 71.9ms
759:	learn: 0.3568127	total: 227ms	remaining: 71.6ms
760:	learn: 0.3556171	total: 227ms	remaining: 71.3ms
761:	learn: 0.3542275	total: 227ms	remaining: 71ms
762:	learn: 0.3529000	total: 227ms	remaining: 70.7ms
763:	learn: 0.3527201	total: 228ms	remaining: 70

In [59]:
pred1 = model_1.predict(X)
pred2 = model_2.predict(X)

In [60]:
predictions = pd.DataFrame({'ID': tab_train['ID'],
                            'pred1': pred1, 'pred2': pred2,
                            'target1': y1['mean_U'], 'target2': y2['mean_V']})

In [61]:
train_blended = train_blended[['ID', 'model_U', 'model_V']]

In [62]:
predictions_blended = pd.merge(predictions, train_blended, 'inner', left_on = 'ID', right_on = 'ID')

In [63]:
predictions_blended['pred1'] = (predictions_blended['pred1']/10 + 9*predictions_blended['model_U']/10)
predictions_blended['pred2'] = (predictions_blended['pred2']/10 + 9*predictions_blended['model_V']/10) 

In [64]:
(sum(abs(predictions_blended['pred1'] - predictions_blended['target1'])/(abs(predictions_blended['pred1']) + abs(predictions_blended['target1']))) +\
sum(abs(predictions_blended['pred2'] - predictions_blended['target2'])/(abs(predictions_blended['pred2']) + abs(predictions_blended['target2']))))/(2*len(predictions_blended))

0.5158754531677225

In [65]:
X = tab_test.copy()
X = X[['nominal_depth', 'horizont_years', 'horizont_months', 'i.mean_U', 'i.mean_V', 'i.std_dev_U', 'i.std_dev_V', 'i.nominal_depth', 'i.mean_cluster_U', 'i.mean_cluster_V', 'sma_U', 'sma_V']]
y1 = tab_test[['mean_U']]
y2 = tab_test[['mean_V']]

In [66]:
pred1 = model_1.predict(X)
pred2 = model_2.predict(X)

In [67]:
predictions = pd.DataFrame({'ID': tab_test['ID'],
                            'pred1': pred1, 'pred2': pred2,
                            'target1': y1['mean_U'], 'target2': y2['mean_V']})

In [68]:
test_blended = test_blended[['ID', 'model_U', 'model_V']]

In [69]:
predictions_blended = pd.merge(predictions, test_blended, 'inner', left_on = 'ID', right_on = 'ID')

In [70]:
predictions_blended['pred1'] = (predictions_blended['pred1']/10 + 9*predictions_blended['model_U']/10)
predictions_blended['pred2'] = (predictions_blended['pred2']/10 + 9*predictions_blended['model_V']/10) 

In [71]:
(sum(abs(predictions_blended['pred1'] - predictions_blended['target1'])/(abs(predictions_blended['pred1']) + abs(predictions_blended['target1']))) +\
sum(abs(predictions_blended['pred2'] - predictions_blended['target2'])/(abs(predictions_blended['pred2']) + abs(predictions_blended['target2']))))/(2*len(predictions_blended))

0.49900644361237395

Как видно, бейзлан пробит, метрика качества на тесте лучше, чем на трейне.

Второй вариант блендинга: блендинг взвешенной модели, EMA и VAR-модели.

In [72]:
%%R
train_blended <- blend_three_models(train_res_weighted, train_res_ema, na.omit(train_res))
calculate_statistics(na.omit(train_blended))

[1] 0.49205


In [73]:
%%R
test_blended <- blend_three_models(test_res_weighted, test_res_ema, na.omit(test_res))
calculate_statistics(na.omit(test_blended))

[1] 0.5076862


Как видно, метрика качества на трейне лучше, чем метрика качества на тесте.

Более того, оба числа примерно одинаковы. Второй блендинг лучше, чем первый.